In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import keras
import tensorflow
from numpy.linalg.linalg import norm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import activations
#from tf_keras_vis.activation_maximization import ActivationMaximization
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, LayerNormalization
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.regularizers import l2
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.markers import MarkerStyle
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import shap
import numpy as np
import cv2


# 画像用
from keras.preprocessing.image import array_to_img, img_to_array, load_img
# モデル読み込み用
from keras.models import load_model

/work/gi55/i55233/miniconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# データの読み込み
data = np.load('/work/gi55/i55233/data/results/bsiso_eeof/prepro_anomaly_8vals.npz')
print('data = ', data.files)

lat = data['lat'][24:49]
lon = data['lon']
olr = data['olr'][80:,24:49,:]
u850 = data['u850'][80:,24:49,:]
v850 = data['v850'][80:,24:49,:]
u200 = data['u200'][80:,24:49,:]
v200 = data['v200'][80:,24:49,:]
h850 = data['h850'][80:,24:49,:]
pr_wtr = data['pr_wtr'][80:,24:49,:]
sst = data['sst'][80:,24:49,:]
time = data['time'][80:]    # 射影後にデータが10日進むため、時刻の方を前進させておく
real_time = pd.to_datetime(time, unit='h', origin=pd.Timestamp('1800-01-01')) # 時刻をdatetime型に変換
print(lat.shape, lon.shape, olr.shape, u850.shape, v850.shape, u200.shape, v200.shape, h850.shape, pr_wtr.shape)
print(real_time[0], real_time[-1])

# 標準化処理
def normalization(data):
  data_mean = np.mean(data, axis=0)
  data_std  = np.std(data, axis=0)
  data_norm = (data - data_mean) / data_std
  print('Raw Data        = ', data.max(), data.min())
  print('Normalized Data = ', data_norm.max(), data_norm.min())
  data_norm = np.nan_to_num(data_norm, nan=0) # 欠損値(nan)を0で置換
  del data_mean, data_std
  return data_norm

olr_norm  = normalization(olr)
u850_norm = normalization(u850)
v850_norm = normalization(v850)
u200_norm = normalization(u200)
v200_norm = normalization(v200)
h850_norm = normalization(h850)
pr_wtr_norm = normalization(pr_wtr)
sst_norm = normalization(sst)

# bsiso index (eEOF) 読み込み
data_file = '/work/gi55/i55233/data/results/bsiso_eeof/bsiso_rt-PCs.npz'
PC      = np.load(data_file)['rt_PCs'][:,:2]
sign    = np.array([-1, 1]).T
PC_norm = sign * PC / PC.std(axis=0)[np.newaxis,:]
time2   = np.load(data_file)['time']
real_time2 = pd.to_datetime(time2, unit='h', origin=pd.Timestamp('1800-01-01')) # 時刻をdatetime型に変換

print('PCs = ', PC_norm.shape)
print('time PCs= ', time2.shape)
print('real time PCs = ', real_time2[0], real_time2[-1])

# インデクシングする関数
def indexing(lead_time):
  output_shape = 2
  rt = real_time2[:-lead_time-1]
  sup_data = PC_norm[lead_time:]
  print(sup_data.shape)
  idx = np.where((rt.year <= 2015))[0]
  sup_train = sup_data[idx]
  idx = np.where((rt.year > 2015))[0]
  sup_test = sup_data[idx]
  print(sup_test.shape, sup_train.shape)
  return data, rt, sup_train, sup_test, output_shape

# 入力データの前処理
def preprocess(data, rt, lead_time):
  ipt_lag0  = data[10:-lead_time-1]
  ipt_lag5  = data[5:-lead_time-6]
  ipt_lag10 = data[:-lead_time-11]

  # 検証データの作成
  idx = np.where((rt.year > 2015))[0]
  ipt_lag0_test = ipt_lag0[idx]
  ipt_lag5_test = ipt_lag5[idx]
  ipt_lag10_test = ipt_lag10[idx]
  ipt_test = np.stack([ipt_lag0_test, ipt_lag5_test, ipt_lag10_test], 3)
  return ipt_test

# ==== iteration program ====
lt_box = [0]
#lt_box = [0, 5, 10, 15, 20, 25, 30, 35]
#lt_box = np.arange(36)
for lead_time in lt_box:

  print('==== lead time : {} day ====='.format(lead_time))

  data, rt, sup_train, sup_test, output_shape = indexing(lead_time) 

  olr_ipt_test = preprocess(olr_norm, rt, lead_time)
  u850_ipt_test = preprocess(u850_norm, rt, lead_time)
  v850_ipt_test = preprocess(v850_norm, rt, lead_time)
  u200_ipt_test = preprocess(u200_norm, rt, lead_time)
  v200_ipt_test = preprocess(v200_norm, rt, lead_time)
  h850_ipt_test = preprocess(h850_norm, rt, lead_time)
  pr_wtr_ipt_test = preprocess(pr_wtr_norm, rt, lead_time)
  sst_ipt_test = preprocess(sst_norm, rt, lead_time)

  ipt_test  = np.concatenate([olr_ipt_test, u850_ipt_test,  u200_ipt_test,
                              v850_ipt_test, v200_ipt_test, h850_ipt_test, 
                              pr_wtr_ipt_test, sst_ipt_test], 3)
  print(ipt_test.shape)


data =  ['lat', 'lon', 'time', 'real_time', 'olr', 'u850', 'v850', 'u200', 'v200', 'h850', 'pr_wtr', 'sst']
(25,) (144,) (15991, 25, 144) (15991, 25, 144) (15991, 25, 144) (15991, 25, 144) (15991, 25, 144) (15991, 25, 144) (15991, 25, 144)
1979-03-22 00:00:00 2022-12-31 00:00:00
Raw Data        =  110.6346296477856 -163.06319252604578
Normalized Data =  8.610244733289218 -15.696228824469003
Raw Data        =  26.628822697430984 -27.39433673019417
Normalized Data =  8.718597179911693 -6.659368241725141
Raw Data        =  19.607371330660023 -21.027599307874095
Normalized Data =  9.371160190483371 -9.169251236677564
Raw Data        =  54.12790452689419 -59.46183253683652
Normalized Data =  6.598683947935241 -8.180679755615591
Raw Data        =  50.62647740730578 -45.80567673624702
Normalized Data =  7.36514770028667 -6.98696916292694
Raw Data        =  132.37072580688695 -196.37969980408127
Normalized Data =  5.4903753314082575 -9.535849081936803
Raw Data        =  36.488246148282585 -32.

/tmp/ipykernel_3488056/527272249.py:24: RuntimeWarning: invalid value encountered in divide
  data_norm = (data - data_mean) / data_std


Raw Data        =  4.588571921210745 -4.560916137984673
Normalized Data =  nan nan
PCs =  (15981, 2)
time PCs=  (15981,)
real time PCs =  1979-04-01 00:00:00 2022-12-31 00:00:00
==== lead time : 0 day =====
(15981, 2)
(2556, 2) (13424, 2)
(2556, 25, 144, 24)


In [16]:
# モデルの読み込み
seed = 000
lead_time = 0
model_path = f'/work/gi55/i55233/data/machine_learning/results/model/model8_test030_wo-v850.h5'
model = load_model(model_path)

TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 64, 64, 8], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']

In [ ]:
datasets = ipt_test[:300]
shap.explainers._deep.deep_tf.op_handlers["FusedBatchNormV3"] = shap.explainers._deep.deep_tf.passthrough # batch norm を挟む場合、このコードが必要：https://github.com/shap/shap/issues/1406
explainer = shap.DeepExplainer(model = model, data = datasets)
shap_values = explainer.shap_values(datasets, check_additivity=False)